## Exercise 5 - Advanced geospatial plotting 

In [1]:
# Import Libraries
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Specify the path to CSV file
file_path = r'C:\Users\north\OneDrive\Dokumente\Career Foundry\Data Visualization 2\Citi-Bike_Bike-Sharing\Data\Prepared Data\merged_citibike_weather.csv'

# Define the required columns as a list
required_columns = [
    'ride_id', 'started_at', 'ended_at', 'start_station_name', 
    'end_station_name', 'start_lat', 'start_lng', 'end_lat', 
    'end_lng', 'start_time', 'end_time', 'date'
]

# Load only the specified columns
citibike_weather_df = pd.read_csv(file_path, usecols=required_columns)

print(citibike_weather_df.head())  # Display the first few rows of the DataFrame

            ride_id               started_at                 ended_at  \
0  70F3BA79664016C1  2022-02-28 09:22:49.821  2022-02-28 09:28:55.174   
1  8FA3412A958122D0  2022-02-14 10:47:30.462  2022-02-14 10:56:04.433   
2  0E2BA02AEAD03EB8  2022-02-18 22:26:55.796  2022-02-18 22:44:31.472   
3  B626F8E1D7ABABBA  2022-02-09 18:38:18.621  2022-02-09 18:47:11.274   
4  605E7CC493D08F3B  2022-02-06 14:52:48.564  2022-02-06 14:55:56.218   

           start_station_name                    end_station_name  start_lat  \
0             9 Ave & W 18 St          Greenwich Ave & Charles St  40.743174   
1    W 120 St & Claremont Ave  Frederick Douglass Blvd & W 117 St  40.810949   
2              E 4 St & 2 Ave          Greenwich Ave & Charles St  40.726281   
3      White St & Johnson Ave              Irving Ave & Harman St  40.707174   
4  S Portland Ave & Hanson Pl         Nevins St & Schermerhorn St  40.685396   

   start_lng    end_lat    end_lng       start_time         end_time  \
0 -74.00

### Data preprocessing

In [3]:
# Add a new column with the value of 1
citibike_weather_df['value'] = 1

# Group by starting and ending stations, and count the trips
aggregated_df = citibike_weather_df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()
aggregated_df.rename(columns={'value': 'trip_count'}, inplace=True)

# Display the first few rows of the aggregated DataFrame
aggregated_df

,start_station_name,end_station_name,trip_count
0,1 Ave & E 110 St,1 Ave & E 110 St,764
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,10
...,...,...,...
1007535,Yankee Ferry Terminal,Water St & Main St,4
1007536,Yankee Ferry Terminal,West St & Chambers St,6
1007537,Yankee Ferry Terminal,West St & Liberty St,4
1007538,Yankee Ferry Terminal,West Thames St,1


In [4]:
print(aggregated_df['trip_count'].sum())
print(citibike_weather_df.shape)

28753448
(28814251, 13)


In [5]:
#Identify rows with missing station names:
missing_start = citibike_weather_df['start_station_name'].isna().sum()
missing_end = citibike_weather_df['end_station_name'].isna().sum()
print(f"Missing start_station_name: {missing_start}")
print(f"Missing end_station_name: {missing_end}")

Missing start_station_name: 49
Missing end_station_name: 60803


In [6]:
#Compare the number of rows with missing station names to the discrepancy:
total_missing = missing_start + missing_end
discrepancy = len(citibike_weather_df) - len(aggregated_df)
print(f"Total missing rows: {total_missing}")
print(f"Discrepancy in trip count: {discrepancy}")

Total missing rows: 60852
Discrepancy in trip count: 27806711


In [7]:
# Check for Rows with Missing Station Names: Ensure there are no duplicates among the rows
missing_rows = citibike_weather_df[citibike_weather_df['start_station_name'].isna() | citibike_weather_df['end_station_name'].isna()]
print(f"Number of missing rows: {len(missing_rows)}")
print(f"Number of unique missing rows: {missing_rows.drop_duplicates().shape[0]}")

Number of missing rows: 60803
Number of unique missing rows: 60803


In [8]:
#Compare Total Rows and Trip Count Compare the total number of trips in citibike_weather_df and the sum of the trip counts in aggregated_df:
aggregated_trip_count = aggregated_df['trip_count'].sum()
print(f"Sum of aggregated trip counts: {aggregated_trip_count}")
print(f"Original dataset size: {len(citibike_weather_df)}")
discrepancy = len(citibike_weather_df) - aggregated_trip_count
print(f"Discrepancy: {discrepancy}")


Sum of aggregated trip counts: 28753448
Original dataset size: 28814251
Discrepancy: 60803


In [9]:
# Check for Duplicate Rows
duplicate_rows = citibike_weather_df.duplicated().sum()
print(f"Number of duplicate rows in the dataset: {duplicate_rows}")

Number of duplicate rows in the dataset: 0


In [10]:
# Identify rows in the original dataset that are excluded during aggregation
unmatched_rows = citibike_weather_df[~citibike_weather_df['start_station_name'].notna() | ~citibike_weather_df['end_station_name'].notna()]
print(unmatched_rows.head())

              ride_id               started_at                 ended_at  \
184  DB5141E47624FAEE  2022-02-06 12:45:51.575  2022-02-06 19:32:27.836   
681  801D69F774F96E7F  2022-02-27 20:49:37.886  2022-02-28 21:49:34.716   
682  E0A1356CAD54CEB2  2022-02-11 09:55:07.738  2022-02-12 10:54:59.211   
688  3BC6F392AAF0E1F8  2022-02-22 16:42:32.804  2022-02-23 17:42:23.131   
691  423E639154B98809  2022-02-18 08:25:29.024  2022-02-19 09:25:06.420   

           start_station_name end_station_name  start_lat  start_lng  end_lat  \
184           8 Ave & W 52 St              NaN  40.763707 -73.985162    40.75   
681     Clinton St & Grand St              NaN  40.715595 -73.987030      NaN   
682        Broadway & W 38 St              NaN  40.752973 -73.987349      NaN   
688     Caton Ave & Argyle Rd              NaN  40.649681 -73.967829      NaN   
691  Southern Blvd & E 174 St              NaN  40.836733 -73.887931      NaN   

     end_lng       start_time         end_time        date  va

In [11]:
# Drop rows with missing values in the 'start_station_name' or 'end_station_name' columns
citibike_weather_df.dropna(subset=['start_station_name', 'end_station_name'], inplace=True)

# Recompute the aggregation
aggregated_df = citibike_weather_df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()
aggregated_df.rename(columns={'value': 'trip_count'}, inplace=True)

# Verify the updated dataset size and aggregation
aggregated_trip_count = aggregated_df['trip_count'].sum()
print(f"Sum of aggregated trip counts after dropping rows: {aggregated_trip_count}")
print(f"Updated dataset size: {len(citibike_weather_df)}")

Sum of aggregated trip counts after dropping rows: 28753448
Updated dataset size: 28753448


#### Handling Missing Values in the Dataset

In the dataset, 60,803 rows (approximately 0.21% of the total dataset) had missing values in the `start_station_name` or `end_station_name` columns. These columns are essential for grouping trips by their starting and ending points, and rows with missing values lack the critical information required for accurate analysis.

To ensure data completeness and reliable results, I decided to drop these rows. The impact of removing this small percentage of data on the overall dataset and analysis is minimal, and it resolves discrepancies in the aggregation process.


In [12]:
aggregated_df['trip_count'].describe()

count    1.007540e+06
mean     2.853827e+01
std      9.589649e+01
min      1.000000e+00
25%      1.000000e+00
50%      4.000000e+00
75%      1.700000e+01
max      1.182900e+04
Name: trip_count, dtype: float64

### Initialize an instance of a kepler.gl map

In [13]:
aggregated_df.head()

,start_station_name,end_station_name,trip_count
0,1 Ave & E 110 St,1 Ave & E 110 St,764
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,10


In [14]:
citibike_weather_df.columns

Index(['ride_id', 'started_at', 'ended_at', 'start_station_name',
       'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'start_time', 'end_time', 'date', 'value'],
      dtype='object')

In [15]:
# Columns to select from citibike_weather_df
columns_weather = ['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']

# Select the required columns and remove duplicates
weather_data = citibike_weather_df[columns_weather].drop_duplicates()

# Merge with trip counts from aggregated_df
df_final = weather_data.merge(aggregated_df, on=['start_station_name', 'end_station_name'], how='inner')

# Display the final DataFrame
print(df_final.head())

  start_station_name            end_station_name  start_lat  start_lng  \
0    9 Ave & W 18 St  Greenwich Ave & Charles St  40.743174 -74.003664   
1    9 Ave & W 18 St  Greenwich Ave & Charles St  40.743174 -74.003664   
2    9 Ave & W 18 St  Greenwich Ave & Charles St  40.743286 -74.003600   
3    9 Ave & W 18 St  Greenwich Ave & Charles St  40.743194 -74.003673   
4    9 Ave & W 18 St  Greenwich Ave & Charles St  40.743251 -74.003636   

     end_lat    end_lng  trip_count  
0  40.735238 -74.000271         142  
1  40.735238 -74.000271         142  
2  40.735238 -74.000271         142  
3  40.735238 -74.000271         142  
4  40.735238 -74.000271         142  


In [16]:
# Save the DataFrame to a CSV file
df_final.to_csv('df_final_with_trip_counts.csv', index=False)

### Plot the map

In [17]:
# Initialize an instance of a kepler.gl map with height and dataset
map = KeplerGl(height=700, data={"data_1": df_final})

# Display the map
map


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':                 start_station_name                end_station_name  start_lat  \
0   …

In [18]:
import pandas as pd
print(pd.__version__)


1.5.1
